# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas e linhas do DataFrame
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None) 

# Base de dados

In [2]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini.csv"

caminho_2 = r"C:\Users\User\OneDrive\Trabalho\ESA 2024\Dashboard_ESA\base_dados\Cel Manfrini(1).csv"
#
#caminho_3 = r"D:\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(2).csv"
#
#caminho_4 = r"D:\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(3).csv"
#
#caminho_5 = r"D:\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(4).csv"
#
#caminho_6 = r"D:\OneDrive\Trabalho\ESA 2023\Dashboard_ESA\base_dados\Cel Manfrini(5).csv"



# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
#df3 = pd.read_csv(caminho_3)
#df4 = pd.read_csv(caminho_4)
#df5 = pd.read_csv(caminho_5)
#df6 = pd.read_csv(caminho_6)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)



# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [3]:
df_saldos = df[['NOME_UG', 'ANO', 'NOME_CREDOR', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [4]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 13.249.574,53
O total de saldo a liquidar pela ESA até o momento foi: R$ 7.715.716,93
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 594.409,59
O total de saldo a pagar pela ESA até o momento foi: R$ 8.310.126,52


In [5]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [6]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [7]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [8]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [9]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [10]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [11]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [12]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

# Suporte para o Dashboard

In [13]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
303,2022NE002482,73594.62,544,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
304,2022NE002483,153862.23,544,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
323,2022NE002702,12733.76,530,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
342,2023NE000034,4172.63,493,PASSAGENS PARA O PAIS,33903301
341,2023NE000033,22498.10,493,PASSAGENS PARA O PAIS,33903301
349,2023NE000157,301.55,481,SERVICOS DOMESTICOS,33903946
343,2023NE000329,2369.34,434,PASSAGENS PARA O PAIS,33903301
311,2023NE000377,480.00,423,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",33903950
795,2023NE000890,478.06,409,SERVICOS DOMESTICOS,33903946
357,2023NE000430,2264.08,405,SERVICOS DOMESTICOS,33903946


In [14]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [15]:
# Qtde de empenhos realizados até o momento
df.shape[0]

1721

# Requisição em API para consulta a CNPJ

In [16]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/03126110000192"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '04/05/1999',
 'atividade_principal': [{'code': '49.30-2-04',
                          'text': 'Transporte rodoviário de mudanças'}],
 'atividades_secundarias': [{'code': '49.30-2-01',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'municipal.'},
                            {'code': '49.30-2-02',
                             'text': 'Transporte rodoviário de carga, exceto '
                                     'produtos perigosos e mudanças, '
                                     'intermunicipal, interestadual e '
                                     'internacional'},
                            {'code': '52.11-7-02', 'text': 'Guarda-móveis'}],
 'bairro': 'ZONA INDUSTRIAL (GUARA)',
 'billing': {'database': True, 'free': True},
 'capital_social': '93700.00',
 'cep': '71.250-710',
 'cnpj': '03.126.110/0001-92',
 'complemento': 'LOTE;',


In [17]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [18]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [19]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [20]:
dicionario['telefone']

'(41) 3316-7700'

In [21]:
dicionario['email']

'fiscal@positivo.com.br'

In [22]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'R JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [23]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['FAV'] != '160129']

df_cnpj['FAV'].unique()

array(['20584944000160', '39750831000116', '05872523000141',
       '17281106000103', '34028316001509', '06981180000116',
       '65295172000185', '02421421000111', '29774797000166',
       '65271868000171', '08403769000162', '42206665000104',
       '49720354000108', '65353401000170', '49577909000104',
       '49834027000179', '11053762000172', '01255291000121',
       '26950671000107', '00139736000145', '14459158000139',
       '40225662000184', '42772140000128', '47217896000129',
       '28719518000107', '01915071000268', '03539933000140',
       '45258432000216', '40695811000179', '30189974000126',
       '15761126000156', '27619368000199', '10942831000136',
       '11202638000121', '15329152000100', '30820280000145',
       '33658130000175', '43892634000109', '20784313000195',
       '28429720000196', '06164562000157', '41841443000192',
       '04191294000137', '20473312000120', '33149502000138',
       '14499338000144', '53202768000104', '18707234000139',
       '36654449000110',

In [24]:
df_cnpj[df_cnpj['FAV'] == '160129']

,UG,NOME_UG,ICFEX,NOME_ICFEX,T_COMANDO,T_RM,T_CIDADE,PROGRAMA,ACAO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,UO,NOME_UO,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [25]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente.head(2)

,UG,NOME_UG,ICFEX,NOME_ICFEX,T_COMANDO,T_RM,T_CIDADE,PROGRAMA,ACAO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,UO,NOME_UO,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
3,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2024NE000004,2024,2024-01-10,160129000012024NE000004,39750831000116,ECOLIMP SERVICOS GERAIS LTDA,RJ,ANO,152,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,52121,COMANDO DO EXERCITO,I3DA,CNT,LICO,26279.57,2656.54,28936.11,74717.49
4,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2024NE000071,2024,2024-02-14,160129000012024NE000071,39750831000116,ECOLIMP SERVICOS GERAIS LTDA,RJ,ANO,117,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,52121,COMANDO DO EXERCITO,I3DA,CNT,LICO,212910.57,0.00,212910.57,0.00


In [26]:
df['ANO'].unique()

array(['2023', '2024', '2022'], dtype=object)

# Pesquisas no DataFrame

In [27]:
# Criando um DataFrame para testes 
df_teste = df.copy()

## Empenhos inscritos em RPNP

In [28]:
# Criando um Dataframe para filtrar os empenhos em RPNP
df_restos = df_teste[df_teste['ANO'].isin(['2022', '2023'])]
df_restos = df_restos[df_restos['A_LIQUIDAR'] !=  0]

### Restos a pagar ND 30 e 52

In [29]:
df_restos_material = df_restos[df_restos['ND'].isin(['339030', '449052' ])]
df_restos_material = df_restos_material.sort_values(by='A_LIQUIDAR', ascending=False)
df_restos_material.head()
df_restos_material.to_excel('restos_a_pagar_2024.xlsx')

In [30]:
cols = ['UG', 'ACAO', 'NOME_ACAO', 'ANO', 'NOME_CREDOR',
'DATA', 'DIAS', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'PI', 
'NOME_PI', 'TIPO', 'TIPO_NE', 'NOME_UGR', 'A_LIQUIDAR', 
'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']

df_filtro = df_teste.loc[:,cols]
df_filtro = df_filtro[df_filtro['A_LIQUIDAR'] != 0]
df_filtro.sort_values(by='DIAS', ascending=False).head(2)

,UG,ACAO,NOME_ACAO,ANO,NOME_CREDOR,DATA,DIAS,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,TIPO,TIPO_NE,NOME_UGR,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
304,160129,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,2022,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-14,544,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002483,B4OMOBMAQUA,AQUARTELAMENTO,RPNP,GLOBAL,DEC-GESTOR,153862.23,1233.96,155096.19,57143.21
303,160129,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,2022,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,2022-12-14,544,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002482,B4OMOBMAQUA,AQUARTELAMENTO,RPNP,GLOBAL,DEC-GESTOR,73594.62,30358.93,103953.55,51843.42


### Filtros Diversos

In [31]:
df_liquidar = df_teste[df_teste['A_LIQUIDAR'] != 0].sort_values(by= 'DIAS', ascending= False).reset_index(drop=True)

In [32]:
df_liquidar.columns

Index(['UG', 'NOME_UG', 'ICFEX', 'NOME_ICFEX', 'T_COMANDO', 'T_RM', 'T_CIDADE',
       'PROGRAMA', 'ACAO', 'NOME_ACAO', 'UGR', 'NOME_UGR', 'PTRES', 'PI',
       'NOME_PI', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'ANO', 'DATA',
       'CHAVE', 'FAV', 'NOME_CREDOR', 'UF_FAV', 'TIPO', 'DIAS', 'UF_UG',
       'FONTE', 'MOD_LIC', 'TIPO_NE', 'RP', 'NOME_RP', 'TIPO_CREDITO',
       'NOME_TIPO_CREDITO', 'PO', 'AUTOR_EMENDA', 'NOME_AUTOR_EMENDA', 'LOCAL',
       'UO', 'NOME_UO', 'RADPI4', 'MEIPI3', 'SUFPI4', 'A_LIQUIDAR',
       'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO'],
      dtype='object')

In [33]:
cols = ['UG', 'ACAO', 'NOME_ACAO', 'NOME_UGR', 'PI',
       'NOME_PI', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'ANO', 'DATA',
       'FAV', 'NOME_CREDOR', 'UF_FAV', 'TIPO', 'DIAS', 'UF_UG',
       'MOD_LIC', 'TIPO_NE', 'RP', 'NOME_RP', 'UO', 'NOME_UO', 'A_LIQUIDAR',
       'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']

df_liquidar.loc[:, cols].head(20)

,UG,ACAO,NOME_ACAO,NOME_UGR,PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,MOD_LIC,TIPO_NE,RP,NOME_RP,UO,NOME_UO,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,DEC-GESTOR,B4OMOBMAQUA,AQUARTELAMENTO,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002483,2022,2022-12-14,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,SP,RPNP,544,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,153862.23,1233.96,155096.19,57143.21
1,160129,219D,ADEQUACAO DE ATIVOS DE INFRAESTRUTURA,DEC-GESTOR,B4OMOBMAQUA,AQUARTELAMENTO,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002482,2022,2022-12-14,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,SP,RPNP,544,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,73594.62,30358.93,103953.55,51843.42
2,160129,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C1ENMILOUTR,OUTROS,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903916,MANUTENCAO E CONSERV. DE BENS IMOVEIS,2022NE002702,2022,2022-12-28,17289559000186,FENARA & FENARA - ENGENHARIA E ADMINISTRACAO LTDA,SP,RPNP,530,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,12733.76,0.00,12733.76,0.00
3,160129,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000033,2023,2023-02-03,14807420000199,FACTO TURISMO LTDA,SP,RPNP,493,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,22498.10,0.00,22498.10,0.00
4,160129,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000034,2023,2023-02-03,21331404000138,ORLEANS VIAGENS E TURISMO LTDA,SP,RPNP,493,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,4172.63,706.91,4879.54,4938.21
5,160129,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903946,SERVICOS DOMESTICOS,2023NE000157,2023,2023-02-15,02347121000210,ASMAR & FURTADO LTDA,MG,RPNP,481,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,301.55,1706.74,2008.29,2038.84
6,160129,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903301,PASSAGENS PARA O PAIS,2023NE000329,2023,2023-04-03,14807420000199,FACTO TURISMO LTDA,SP,RPNP,434,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,2369.34,0.00,2369.34,1789.14
7,160129,21A0,APRESTAMENTO DAS FORCAS,COEX/GESTOR,E6RVPLJMTOE,MEDIDAS PROFILÁT SAÚDE EQUINOS,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE000377,2023,2023-04-14,08593332000139,EQUALLI SERVICOS VETERINARIOS LTDA,SP,RPNP,423,MG,DISPENSA DE LICITACAO,ORDINARIO,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,480.00,0.00,480.00,0.00
8,167129,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,DGP,D8SACNTLARO,LAVAGEM DE ROUPAS PARA A VIDA VEGETATIVA OM,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903946,SERVICOS DOMESTICOS,2023NE000890,2023,2023-04-28,02347121000210,ASMAR & FURTADO LTDA,MG,RPNP,409,MG,PREGAO,GLOBAL,1,PRIMARIO OBRIGATORIO,52921,FUNDO DO EXERCITO,478.06,0.00,478.06,2921.31
9,160129,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,DECEX/GESTOR,C4ENMILOUTR,RESERVA (DETMIL),339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903946,SERVICOS DOMESTICOS,2023NE000430,2023,2023-05-02,02347121000210,ASMAR & FURTADO LTDA,MG,RPNP,405,MG,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,52121,COMANDO DO EXERCITO,2264.08,0.00,2264.08,7277.68


In [34]:
df_liquidar.to_excel('empenhos_2024_liquidar.xlsx')

In [35]:
df_teste['A_LIQUIDAR'].sum().round(2)


7715716.93

# SIPO 2024

In [36]:
df_sipo = df_teste[df_teste['PI'] == 'C4ENMILCAPE']

In [37]:
df_sipo.to_excel('empenhos_sipo.xlsx')

In [39]:
df_teste.columns

Index(['UG', 'NOME_UG', 'ICFEX', 'NOME_ICFEX', 'T_COMANDO', 'T_RM', 'T_CIDADE',
       'PROGRAMA', 'ACAO', 'NOME_ACAO', 'UGR', 'NOME_UGR', 'PTRES', 'PI',
       'NOME_PI', 'ND', 'NOME_ND', 'NDSI', 'NOME_NDSI', 'NE', 'ANO', 'DATA',
       'CHAVE', 'FAV', 'NOME_CREDOR', 'UF_FAV', 'TIPO', 'DIAS', 'UF_UG',
       'FONTE', 'MOD_LIC', 'TIPO_NE', 'RP', 'NOME_RP', 'TIPO_CREDITO',
       'NOME_TIPO_CREDITO', 'PO', 'AUTOR_EMENDA', 'NOME_AUTOR_EMENDA', 'LOCAL',
       'UO', 'NOME_UO', 'RADPI4', 'MEIPI3', 'SUFPI4', 'A_LIQUIDAR',
       'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO'],
      dtype='object')

In [40]:
df_teste.head()

,UG,NOME_UG,ICFEX,NOME_ICFEX,T_COMANDO,T_RM,T_CIDADE,PROGRAMA,ACAO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,UO,NOME_UO,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000661,2023,2023-06-14,160129000012023NE000661,20584944000160,TBL SERVICOS LTDA,MG,RPNP,362,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,52121,COMANDO DO EXERCITO,I3DA,CNT,LICO,0.00,0.00,0.00,72837.98
1,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000849,2023,2023-07-05,160129000012023NE000849,20584944000160,TBL SERVICOS LTDA,MG,RPNP,341,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,52121,COMANDO DO EXERCITO,I3DA,CNT,LICO,0.00,0.00,0.00,36418.99
2,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2023NE000994,2023,2023-08-15,160129000012023NE000994,20584944000160,TBL SERVICOS LTDA,MG,RPNP,300,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,52121,COMANDO DO EXERCITO,I3DA,CNT,LICO,0.00,0.00,0.00,36418.99
3,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2024NE000004,2024,2024-01-10,160129000012024NE000004,39750831000116,ECOLIMP SERVICOS GERAIS LTDA,RJ,ANO,152,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,52121,COMANDO DO EXERCITO,I3DA,CNT,LICO,26279.57,2656.54,28936.11,74717.49
4,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,0032,2000,ADMINISTRACAO DA UNIDADE,160073,DGO,171460,I3DACNTLICO,LIMPEZA DE BENS IMÓVEIS,339037,LOCACAO DE MAO-DE-OBRA,33903702,LIMPEZA E CONSERVACAO,2024NE000071,2024,2024-02-14,160129000012024NE000071,39750831000116,ECOLIMP SERVICOS GERAIS LTDA,RJ,ANO,117,MG,1000000000,PREGAO,GLOBAL,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0004,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS - AV. SETE DE SE...,52121,COMANDO DO EXERCITO,I3DA,CNT,LICO,212910.57,0.00,212910.57,0.00


In [42]:
df_teste[df_teste['NE'] == '2023NE002715']

,UG,NOME_UG,ICFEX,NOME_ICFEX,T_COMANDO,T_RM,T_CIDADE,PROGRAMA,ACAO,NOME_ACAO,UGR,NOME_UGR,PTRES,PI,NOME_PI,ND,NOME_ND,NDSI,NOME_NDSI,NE,ANO,DATA,CHAVE,FAV,NOME_CREDOR,UF_FAV,TIPO,DIAS,UF_UG,FONTE,MOD_LIC,TIPO_NE,RP,NOME_RP,TIPO_CREDITO,NOME_TIPO_CREDITO,PO,AUTOR_EMENDA,NOME_AUTOR_EMENDA,LOCAL,UO,NOME_UO,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
333,160129,ESA,160533,4º CGCFEX,CML,4ª RM,Três Corações,6012,8965,CAPACITACAO PROFISSIONAL MILITAR DO E,160503,DECEX/GESTOR,171423,C4ENMILCAPE,CAPACITACAO DE PESSOAL,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE002715,2023,2023-12-19,160129000012023NE002715,49577909000104,COMERCIAL RB - COMERCIO VAREJISTA E ATACADISTA...,SP,RPNP,174,MG,1000000000,PREGAO,ORDINARIO,2,PRIMARIO DISCRICIONARIO,A,INICIAL (LOA),0000,000000000000,SEM EMENDA,ESCOLA DE SARGENTOS DAS ARMAS,52121,COMANDO DO EXERCITO,C4EN,MIL,CAPE,1340.6,0.0,1340.6,0.0
